# 🚀 Production Zoho Data Sync Runner

**The "Cockpit" Notebook - Clean Production Orchestration**

This notebook provides a clean, linear interface for running the complete Zoho Data Sync pipeline in production. It imports directly from the production `src/` package and executes the comprehensive sync workflow with full reporting.

## Features
- **Production-Ready Execution**: Complete JSON-to-database sync
- **Enhanced Field Mapping**: Near-100% field coverage using enhanced mappings
- **Comprehensive Verification**: End-to-end data verification and reporting
- **Multi-Format Reports**: JSON, CSV, and Markdown reports
- **Clean Architecture**: Minimal logic, maximum orchestration

## Workflow
1. **Configuration & Setup**: Load configuration and initialize components
2. **Data Source Analysis**: Analyze JSON files and database state
3. **Import Plan Generation**: Create intelligent import strategy
4. **Sync Execution**: Execute comprehensive sync with progress tracking
5. **Verification & Reporting**: Verify results and generate comprehensive reports
6. **Success Summary**: Display final results and recommendations

## 1. Configuration & Setup

Initialize the production environment and load configuration.

In [ ]:
# Production imports from src package
import sys
from pathlib import Path
import logging
from datetime import datetime

# Setup project path
notebook_dir = Path.cwd()
if notebook_dir.name == "notebooks":
    project_root = notebook_dir.parent
else:
    project_root = notebook_dir

# Add src to Python path
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Production imports
from data_pipeline import (
    ComprehensiveSyncManager,
    ConfigurationManager,
    generate_comparison_report,
    create_comprehensive_sync_manager,
    execute_production_sync
)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('production_sync')

print("🚀 Production Zoho Data Sync Runner Initialized")
print(f"📁 Project Root: {project_root}")
print(f"📦 Source Path: {src_path}")
print(f"⏰ Session Start: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("✅ Ready for production sync execution")

## 2. Environment Configuration

Load and validate the production configuration.

In [ ]:
# Load production configuration
config = ConfigurationManager()

# Get key configuration paths
json_base_path = config.get('data_sources', 'json_api_base_path')
database_path = config.get('data_sources', 'target_database')

# Resolve to absolute paths
json_directory = project_root / json_base_path
db_path = project_root / database_path

print("⚙️ Production Configuration Loaded")
print(f"📂 JSON Directory: {json_directory}")
print(f"🗄️ Database Path: {db_path}")
print(f"✅ Configuration Valid: {json_directory.exists() and db_path.parent.exists()}")

# Validate paths
if not json_directory.exists():
    print(f"⚠️ WARNING: JSON directory does not exist: {json_directory}")
if not db_path.parent.exists():
    print(f"⚠️ WARNING: Database directory does not exist: {db_path.parent}")

print("\n📋 Configuration Summary:")
for section, values in config.config.items():
    print(f"  [{section}]")
    for key, value in values.items():
        print(f"    {key}: {value}")

## 3. Quick Data Overview

Generate a quick comparison report to understand the current state.

In [ ]:
# Generate quick comparison report
print("🔍 Generating Quick Data Overview...")

comparison_df = generate_comparison_report(
    database_path=str(db_path),
    json_directory=str(json_directory)
)

if not comparison_df.empty:
    print("\n📊 JSON vs Database Comparison:")
    print("=" * 80)
    
    # Display formatted comparison
    for _, row in comparison_df.iterrows():
        entity = row['entity_name']
        json_count = row['json_records']
        db_count = row['database_records']
        status = row['status']
        icon = row['status_icon']
        sync_pct = row['sync_percentage']
        
        print(f"{icon} {entity:<18} │ JSON: {json_count:>6,} │ DB: {db_count:>6,} │ {sync_pct:>5.1f}% │ {status}")
    
    # Summary stats
    total_json = comparison_df['json_records'].sum()
    total_db = comparison_df['database_records'].sum()
    avg_sync = comparison_df['sync_percentage'].mean()
    
    print("\n" + "=" * 80)
    print(f"📈 Total JSON Records: {total_json:,}")
    print(f"🗄️ Total DB Records: {total_db:,}")
    print(f"🎯 Average Sync Rate: {avg_sync:.1f}%")
    
    # Identify entities needing sync
    needs_sync = comparison_df[comparison_df['sync_percentage'] < 100]
    if not needs_sync.empty:
        print(f"\n⚠️ Entities needing sync: {len(needs_sync)}")
        for _, row in needs_sync.iterrows():
            print(f"   {row['status_icon']} {row['entity_name']}: {row['status']}")
    else:
        print("\n✅ All entities appear to be synchronized")
        
else:
    print("❌ No comparison data available - check JSON directory and database paths")

## 4. Execute Comprehensive Sync

Run the complete production sync workflow with enhanced mappings and comprehensive verification.

In [ ]:
# Execute comprehensive production sync
print("🚀 Executing Comprehensive Production Sync")
print("=" * 60)

# Create comprehensive sync manager
sync_manager = create_comprehensive_sync_manager()

# Execute the complete workflow
workflow_results = sync_manager.execute_comprehensive_sync(
    json_directory=str(json_directory),
    generate_reports=True,
    auto_confirm=True  # Production mode - auto-confirm reasonable operations
)

print("\n📊 Sync Workflow Completed")
print(f"✅ Overall Success: {workflow_results['success']}")
print(f"📋 Session ID: {workflow_results['session_id']}")

# Store results for detailed analysis
production_sync_results = workflow_results

## 5. Sync Results Analysis

Analyze the detailed results of the sync operation.

In [ ]:
# Analyze sync results
print("📈 Production Sync Results Analysis")
print("=" * 50)

if 'production_sync_results' in locals():
    results = production_sync_results
    
    # Analysis phase results
    analysis = results.get('analysis', {})
    if analysis.get('success'):
        print(f"\n📊 Data Analysis:")
        print(f"   Entities found: {len(analysis.get('entities_found', []))}")
        print(f"   Total files: {analysis.get('total_files', 0)}")
        print(f"   Total records: {analysis.get('total_records', 0):,}")
        
        # Show entity details
        for entity in analysis.get('entities_found', []):
            details = analysis.get('file_details', {}).get(entity, {})
            record_count = details.get('record_count', 0)
            custom_fields = len(details.get('custom_fields', []))
            print(f"   📋 {entity}: {record_count:,} records, {custom_fields} custom fields")
    
    # Import execution results
    execution = results.get('execution', {})
    if execution:
        print(f"\n⚡ Import Execution:")
        print(f"   Entities processed: {execution.get('entities_processed', 0)}")
        print(f"   Entities successful: {execution.get('entities_successful', 0)}")
        print(f"   Entities failed: {execution.get('entities_failed', 0)}")
        print(f"   Total records imported: {execution.get('total_records_imported', 0):,}")
        
        # Show any errors
        errors = execution.get('errors', [])
        if errors:
            print(f"\n❌ Import Errors ({len(errors)}):")
            for error in errors[:5]:  # Show first 5 errors
                print(f"   - {error}")
    
    # Verification results
    verification = results.get('verification', {})
    if verification:
        print(f"\n✅ Verification Results:")
        print(f"   Entities verified: {verification.get('entities_verified', 0)}")
        print(f"   Entities passed: {verification.get('entities_passed', 0)}")
        print(f"   Overall success rate: {verification.get('overall_success_rate', 0):.1f}%")
        
        # Show verification details
        details = verification.get('verification_details', {})
        for entity, verify_info in details.items():
            expected = verify_info.get('expected_records', 0)
            actual = verify_info.get('actual_records', 0)
            match_pct = verify_info.get('match_percentage', 0)
            passed = verify_info.get('passed', False)
            status_icon = "✅" if passed else "❌"
            print(f"   {status_icon} {entity}: {actual:,}/{expected:,} records ({match_pct:.1f}%)")
    
    # Reports generated
    reports = results.get('reports', {})
    if reports.get('generation_success'):
        print(f"\n📄 Reports Generated:")
        if reports.get('json_report_path'):
            print(f"   📊 JSON Report: {Path(reports['json_report_path']).name}")
        if reports.get('csv_report_path'):
            print(f"   📈 CSV Report: {Path(reports['csv_report_path']).name}")
        if reports.get('markdown_report_path'):
            print(f"   📝 Markdown Report: {Path(reports['markdown_report_path']).name}")
    
else:
    print("❌ No sync results available - please run the sync workflow first")

## 6. Final Production Summary

Display the final production summary with recommendations.

In [ ]:
# Final production summary
print("🎯 FINAL PRODUCTION SYNC SUMMARY")
print("=" * 60)

if 'production_sync_results' in locals() and production_sync_results:
    results = production_sync_results
    summary = results.get('summary', {})
    
    # Overall status
    overall_success = results.get('success', False)
    success_icon = "🎉" if overall_success else "⚠️"
    
    print(f"\n{success_icon} SYNC STATUS: {'SUCCESS' if overall_success else 'COMPLETED WITH ISSUES'}")
    print(f"📋 Session ID: {results.get('session_id', 'N/A')}")
    print(f"⏰ Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Key metrics
    analysis = results.get('analysis', {})
    execution = results.get('execution', {})
    verification = results.get('verification', {})
    
    print(f"\n📊 KEY METRICS:")
    print(f"   📁 Entities Analyzed: {len(analysis.get('entities_found', []))}")
    print(f"   ⚡ Entities Imported: {execution.get('entities_successful', 0)}")
    print(f"   📈 Records Imported: {execution.get('total_records_imported', 0):,}")
    print(f"   ✅ Verification Rate: {verification.get('overall_success_rate', 0):.1f}%")
    print(f"   📄 Reports Generated: {'Yes' if results.get('reports', {}).get('generation_success') else 'No'}")
    
    # Recommendations
    recommendations = summary.get('recommendations', results.get('recommendations', []))
    if recommendations:
        print(f"\n💡 RECOMMENDATIONS:")
        for rec in recommendations:
            print(f"   • {rec}")
    
    # Next steps
    print(f"\n🚀 NEXT STEPS:")
    if overall_success:
        print(f"   ✅ Sync completed successfully - data is ready for use")
        print(f"   📊 Review generated reports for detailed analysis")
        print(f"   🔄 Schedule regular sync operations as needed")
    else:
        print(f"   ⚠️ Review error logs and verification results")
        print(f"   🔧 Address any identified issues before next sync")
        print(f"   📞 Contact support if issues persist")
    
    # Production readiness
    verification_rate = verification.get('overall_success_rate', 0)
    if verification_rate >= 95.0 and overall_success:
        print(f"\n🏆 PRODUCTION STATUS: READY")
        print(f"   🎯 High verification success rate ({verification_rate:.1f}%)")
        print(f"   ✅ All critical workflows completed successfully")
        print(f"   📊 Comprehensive reports available for stakeholders")
    else:
        print(f"\n⚠️ PRODUCTION STATUS: NEEDS ATTENTION")
        print(f"   📉 Verification rate below threshold ({verification_rate:.1f}% < 95%)")
        print(f"   🔧 Review and resolve issues before production deployment")

else:
    print("❌ No sync results available")
    print("🔧 Please run the comprehensive sync workflow first")

print(f"\n" + "=" * 60)
print(f"🏁 Production Sync Runner Session Complete")
print(f"⏰ {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")